# Whole drive map

### Imports & external variables.

In [1]:
import os
import csv
import pandas as pd

target_drive = "Unit_SCT"
base_path="Y:\\"
output_path="C:\\Users\\manakang\\Videos\\Repos\\papermill\\outputs\\"
interesting_file_types = ["doc","docx","txt","pdf","xlsx"]
data_file_identifiers = ["Data","data analyse","Environmental Data","data","data_analyse","Environmental_Data","environmental data"]
other_file_identifiers = ["07 Other","5. Gerelateerde documenten","5 Geralateerde documenten","Gerelateerde documenten","08 KLAD","08 Metadata Fiches","9. Citizen science","fact sheet","5. Faseplan","8. Faseplan"]
output_file_identifiers = ["7. WO","4. Wetenschappelijke Output","4 Wetenschappelijke output","06 Reports_scientific output","4. WO","4WO","4 WO","Wetenschappelijke output","WO","Resultaten"]
report_file_identifiers = ["Rapportering","rapportering","Rapporten","rapporten","Reporting","reporting","Rapport","Rapportage","rapportage","rapport"]
work_file_identifiers = ["Werkdocumenten","werkdocumenten","Workdoc_Results","Workdocs","workdocs","workdoc","Workdoc"]
project_file_identifiers = ["projectbeheer","Projectbeheer","projectmanagement","Projectmanagement","beheer","Beheer","administration","Administration","Administratie","administratie",]

In [2]:
base_path = base_path+target_drive
all_files_save = output_path+"drive_maps_"+target_drive+".csv"
interesting_files_save = output_path+"/interesting_files_"+target_drive+".csv"
project_files_save = output_path+"/project_files_"+target_drive+".csv"
files_list = []
project_files = []
non_project_files = []

print(target_drive)
print(base_path)
print(output_path)
print(all_files_save)
print(interesting_files_save)
print(project_files_save)

Unit_SCT
Y:\Unit_SCT
C:\Users\manakang\Videos\Repos\papermill\outputs\
C:\Users\manakang\Videos\Repos\papermill\outputs\drive_maps_Unit_SCT.csv
C:\Users\manakang\Videos\Repos\papermill\outputs\/interesting_files_Unit_SCT.csv
C:\Users\manakang\Videos\Repos\papermill\outputs\/project_files_Unit_SCT.csv


### Getting all the files
Using os.walk to get all the files and their paths. \\
Storing the path and the file name in the array seperately in a tuple

Reading the files from csv into a dataframe \
This is done because there are just too many files to practically scrape the entiredrive.

In [3]:
unit_sct_df = pd.read_csv(all_files_save)

Splitting path in individual folders

In [ ]:
unit_sct_df['splitpath'] = unit_sct_df['path']
unit_sct_df['splitpath'] = unit_sct_df['splitpath'].str.split(pat="\\")
# print(unit_sct_df.tail)

Getting the "depth" of the file path

In [ ]:
unit_sct_df['Length'] = unit_sct_df['splitpath'].map(len)
# print(unit_sct_df['Length'][unit_sct_df.index[-1]])
# print(unit_sct_df['Length'].max())

Splitting the file to filename and file type

In [ ]:
unit_sct_df['splitfile'] = unit_sct_df['file']
unit_sct_df['splitfile'] = unit_sct_df['splitfile'].str.split(pat=".")
# print(unit_sct_df['splitfile'][unit_sct_df.index[-1]])
# print(unit_sct_df.tail)

keeping only relevant file types

In [ ]:
unit_sct_df = unit_sct_df.loc[unit_sct_df.splitfile.str[-1].isin(interesting_file_types)]
# print(unit_sct_df.tail)

In [ ]:
unit_sct_df = unit_sct_df.loc[unit_sct_df.splitfile.str[-1].isin(interesting_file_types)]
# print(unit_sct_df.tail)

Trying to find projects\
Looking for project in the folder path\
and then looking for files with more than 6 digits in the name

In [ ]:
# custom function to count digits
def count_digits(string):
    # cheking if enough digits
    numbers = sum(c.isdigit() for c in string)
    if numbers < 6:
        return False
    # checking if digits are sequential
    counter = 0
    for c in string:
        if c.isdigit():
            counter += 1
            if counter >= 6:
                return True
        else:
            counter = 0 
    return False

# looking for project keywords in folder path
new_dataframe = unit_sct_df.loc[unit_sct_df.path.str.contains('|'.join(project_file_identifiers))]
# counting digits in the file name
new_dataframe = new_dataframe.loc[new_dataframe.file.apply(count_digits)]
# creating excel
with pd.ExcelWriter('results.xlsx') as writer:  
    new_dataframe.to_excel(writer, sheet_name='project_files', encoding='utf-8')

Looking for other kinds of files

In [ ]:
# Finding data files
def files_to_excel(file_type,identifiers):
    # find the files
    new_sheet_name = str(file_type)+'_files'
    new_dataframe = unit_sct_df.loc[unit_sct_df.file.str.contains('|'.join(identifiers))]
    print(new_dataframe.shape)
    # add sheet
    with pd.ExcelWriter(output_file, mode='a', engine='openpyxl' ,engine_kwargs={'encoding':'utf-8'}) as writer:  
        new_dataframe.to_excel(writer, sheet_name=new_sheet_name)
        

files_to_excel('data',data_file_identifiers)
files_to_excel('other',other_file_identifiers)
files_to_excel('output',output_file_identifiers)
files_to_excel('report',report_file_identifiers)
files_to_excel('work',work_file_identifiers)
files_to_excel('project',project_file_identifiers)